In [55]:
import arcpy
from arcpy.sa import *
import os
import glob
import arcpy
import os
import glob
import numpy as np
#def run_the_script(BASE_PATH):
###### THIS CODE reads the results of krigging wells and put them the extent of the DEM map that we used for MODFLOW modeling. 

arcpy.env.overwriteOutput=True
BASE_PATH=f'D:/GW_data_interpolated'
SOURCE_mask="DEM/DEM_250m.tif"

#Input_rasters="R:/ram_disk_workspace"

Input_rasters= r"Well_data_krigging\Krigging_work_space_250m_entire_michigan\Krigging_work_space"

MASK_shape=os.path.join(BASE_PATH, SOURCE_mask)

OUTPUT_path = os.path.join(BASE_PATH, '_temp')

arcpy.env.overwriteOutput=True
FILES=glob.glob(os.path.join(BASE_PATH,Input_rasters+'/*.tif'))

SOURCE_extent_raster = os.path.join(BASE_PATH, "DEM/DEM_250m.tif")

workspace=os.path.join(BASE_PATH,Input_rasters)
arcpy.env.workspace= workspace
dem_raster = arcpy.Raster(SOURCE_extent_raster)

desc = arcpy.Describe(dem_raster)
cell_size = desc.meanCellWidth  # or desc.meanCellHeight, they are usually the same


arcpy.env.outputCoordinateSystem = arcpy.Describe(SOURCE_extent_raster).spatialReference

for FILE in FILES:

    # Clip by extent
    TEMP_name = os.path.basename(FILE)
    temp_file_path = os.path.join(OUTPUT_path, TEMP_name)
    dem_extent = dem_raster.extent
    arcpy.Clip_management(aligned_raster_path, f"{dem_extent.XMin} {dem_extent.YMin} {dem_extent.XMax} {dem_extent.YMax}", 
                          temp_file_path, MASK_shape, "#", "NONE", "NO_MAINTAIN_EXTENT")

    
    
    # Clip the raster first
    arcpy.Clip_management(aligned_raster_path, f"{dem_extent.XMin} {dem_extent.YMin} {dem_extent.XMax} {dem_extent.YMax}", 
                          temp_file_path, MASK_shape, "#", "NONE", "NO_MAINTAIN_EXTENT")

    # Now resample the raster to the desired dimensions
    desired_width = 1458
    desired_height = 1849
    # Create an empty raster with the desired dimensions
    empty_raster_path = os.path.join(OUTPUT_path, "empty_raster.tif")
    spatial_ref = arcpy.Describe(dem_raster).spatialReference

    # Create an empty raster with the desired dimensions
    arcpy.CreateRasterDataset_management(
        OUTPUT_path, 
        "empty_raster.tif", 
        cellsize=cell_size, 
        pixel_type="32_BIT_FLOAT", 
        raster_spatial_reference=spatial_ref, 
        number_of_bands=1
    )
    
    
    arcpy.env.extent = arcpy.Describe(SOURCE_extent_raster).extent

    # Debugging info
    print("New extent set to: ", arcpy.env.extent)
    # Calculate statistics for the empty raster
    arcpy.CalculateStatistics_management(empty_raster_path)

    # Calculate statistics for the clipped raster as well
    arcpy.CalculateStatistics_management(temp_file_path)
    
    arcpy.DefineProjection_management(empty_raster_path, arcpy.Describe(SOURCE_extent_raster).spatialReference)

    # Perform a conditional operation to overlay the clipped raster onto the empty raster
    result_raster_path = os.path.join(OUTPUT_path, "final_" + TEMP_name)
    # After clipping, project the raster to the desired CRS
    projected_temp_file_path = os.path.join(OUTPUT_path, "projected_" + TEMP_name)
    arcpy.ProjectRaster_management(temp_file_path, projected_temp_file_path, arcpy.Describe(SOURCE_extent_raster).spatialReference)

    # Perform a conditional operation using the projected raster
    arcpy.gp.Con_sa(empty_raster_path, projected_temp_file_path, result_raster_path, "", "VALUE IS NULL")



New extent set to:  3794815.7051 21639.1843 4159315.7051 483889.1843 NaN NaN NaN NaN
New extent set to:  3794815.7051 21639.1843 4159315.7051 483889.1843 NaN NaN NaN NaN
New extent set to:  3794815.7051 21639.1843 4159315.7051 483889.1843 NaN NaN NaN NaN
New extent set to:  3794815.7051 21639.1843 4159315.7051 483889.1843 NaN NaN NaN NaN
New extent set to:  3794815.7051 21639.1843 4159315.7051 483889.1843 NaN NaN NaN NaN
New extent set to:  3794815.7051 21639.1843 4159315.7051 483889.1843 NaN NaN NaN NaN
New extent set to:  3794815.7051 21639.1843 4159315.7051 483889.1843 NaN NaN NaN NaN
New extent set to:  3794815.7051 21639.1843 4159315.7051 483889.1843 NaN NaN NaN NaN
New extent set to:  3794815.7051 21639.1843 4159315.7051 483889.1843 NaN NaN NaN NaN
New extent set to:  3794815.7051 21639.1843 4159315.7051 483889.1843 NaN NaN NaN NaN
New extent set to:  3794815.7051 21639.1843 4159315.7051 483889.1843 NaN NaN NaN NaN
New extent set to:  3794815.7051 21639.1843 4159315.7051 483889.1

In [49]:
SOURCE_extent_raster

'D:/GW_data_interpolated\\DEM/DEM_250m.tif'

In [48]:
temp_file_path

'D:/GW_data_interpolated\\_temp\\kriging_output_AQ_THK_1.tif'

In [47]:
# Debugging info
print("Spatial Reference of empty raster: ", arcpy.Describe(empty_raster_path).spatialReference.name)
print("Spatial Reference of temp raster: ", arcpy.Describe(temp_file_path).spatialReference.name)
print("Extent of empty raster: ", arcpy.Describe(empty_raster_path).extent)
print("Extent of temp raster: ", arcpy.Describe(temp_file_path).extent)
# Set the extent to match the DEM



Spatial Reference of empty raster:  NAD_1983_StatePlane_Michigan_South_FIPS_2113
Spatial Reference of temp raster:  NAD_1983_StatePlane_Michigan_South_FIPS_2113
Extent of empty raster:  41 -219 291 31 NaN NaN NaN NaN
Extent of temp raster:  3845533.96828974 48731.4983070933 4159283.96828974 483981.498307093 NaN NaN NaN NaN
New extent set to:  3794815.7051 21639.1843 4159315.7051 483889.1843 NaN NaN NaN NaN


ExecuteError: ERROR 010423: D:\GW_data_interpolated\_temp\empty_raster.tif does not have valid statistics as required by the operation.
ERROR 010423: D:\GW_data_interpolated\_temp\empty_raster.tif does not have valid statistics as required by the operation.
Failed to execute (Con).


In [52]:
empty_raster_path

'D:/GW_data_interpolated\\_temp\\empty_raster.tif'

In [45]:
spatial_ref

name (Projected Coordinate System),NAD_1983_StatePlane_Michigan_South_FIPS_2113
factoryCode (WKID),26990
linearUnitName (Linear Unit),Meter
name (Geographic Coordinate System),GCS_North_American_1983
factoryCode (WKID),4269
angularUnitName (Angular Unit),Degree
datumName (Datum),D_North_American_1983


In [38]:
import rasterio
import numpy as np
from rasterio.enums import Resampling
from rasterio.windows import Window



arcpy.env.overwriteOutput=True
BASE_PATH=f'D:/GW_data_interpolated'
SOURCE_mask="DEM/DEM_250m.tif"

#Input_rasters="R:/ram_disk_workspace"

Input_rasters= r"Well_data_krigging\Krigging_work_space_250m_entire_michigan\Krigging_work_space"

MASK_shape=os.path.join(BASE_PATH, SOURCE_mask)

temp_path = os.path.join(BASE_PATH, '_temp/_temp')
os.makedirs(temp_path, exist_ok=True)

arcpy.env.overwriteOutput=True
FILES=glob.glob(os.path.join(BASE_PATH,Input_rasters+'/*.tif'))
DEM_path = os.path.join(BASE_PATH, "DEM/DEM_250m.tif")
path_to_files=os.path.join(BASE_PATH,Input_rasters)

OUTPUT_path = os.path.join(BASE_PATH, '_temp')
# Load the DEM to get its metadata
with rasterio.open(DEM_path) as dem_src:
    dem_meta = dem_src.meta
    dem_array = dem_src.read(1)  # Assuming the DEM is a single-band raster

# Iterate through the raster files
for file_path in FILES:
    with rasterio.open(file_path) as src:
        # Read the raster data
        array = src.read(1)
        
        # Add 100 rows and columns as a buffer
        new_shape = (array.shape[0] + 200, array.shape[1] + 200)
        new_array = np.full(new_shape, src.nodatavals[0], dtype=array.dtype)  # Using the NoData value from the original raster
        new_array[100:-100, 100:-100] = array

        # Create a new transform for the new array
        new_transform = rasterio.Affine.translation(-100 * dem_meta['transform'][0], -100 * dem_meta['transform'][4]) * src.transform
        
        # Prepare meta data for new raster
        new_meta = src.meta.copy()
        new_meta.update({
            'height': new_array.shape[0],
            'width': new_array.shape[1],
            'transform': new_transform
        })
        
        # Write the new raster file
        with rasterio.open(temp_path+os.path.basename(file_path), 'w', **new_meta) as new_dst:
            new_dst.write(new_array, 1)
            
    # Reopen the new raster file to clip it based on the DEM
    with rasterio.open(temp_path+os.path.basename(file_path)) as src:
        # Create a window based on the extent of the DEM
        window = rasterio.windows.from_bounds(
            dem_meta['transform'][2], dem_meta['transform'][5] + dem_meta['height'] * dem_meta['transform'][0],
            dem_meta['transform'][2] + dem_meta['width'] * dem_meta['transform'][0], dem_meta['transform'][5],
            transform=src.transform
        )
        
        # Read the window
        array = src.read(1, window=window)
        
        # Update metadata
        new_meta = src.meta.copy()
        new_meta.update({
            'height': array.shape[0],
            'width': array.shape[1],
            'transform': rasterio.windows.transform(window, src.transform)
        })
        
        # Write the clipped raster
        with rasterio.open(OUTPUT_path+os.path.basename(file_path), 'w', **new_meta) as new_dst:
            new_dst.write(array, 1)


WindowError: Bounds and transform are inconsistent

In [37]:
FILES

['D:/GW_data_interpolated\\Well_data_krigging\\Krigging_work_space_250m_entire_michigan\\Krigging_work_space\\kriging_output_AQ_THK_1.tif',
 'D:/GW_data_interpolated\\Well_data_krigging\\Krigging_work_space_250m_entire_michigan\\Krigging_work_space\\kriging_output_AQ_THK_2.tif',
 'D:/GW_data_interpolated\\Well_data_krigging\\Krigging_work_space_250m_entire_michigan\\Krigging_work_space\\kriging_output_H_COND_1.tif',
 'D:/GW_data_interpolated\\Well_data_krigging\\Krigging_work_space_250m_entire_michigan\\Krigging_work_space\\kriging_output_H_COND_2.tif',
 'D:/GW_data_interpolated\\Well_data_krigging\\Krigging_work_space_250m_entire_michigan\\Krigging_work_space\\kriging_output_SWL.tif',
 'D:/GW_data_interpolated\\Well_data_krigging\\Krigging_work_space_250m_entire_michigan\\Krigging_work_space\\kriging_output_TRANSMSV_1.tif',
 'D:/GW_data_interpolated\\Well_data_krigging\\Krigging_work_space_250m_entire_michigan\\Krigging_work_space\\kriging_output_TRANSMSV_2.tif',
 'D:/GW_data_interpol

In [13]:
OUTPUT_path

'D:/GW_data_interpolated\\_temp'

In [24]:
FILES

['D:/GW_data_interpolated\\Well_data_krigging\\Krigging_work_space_250m_entire_michigan\\Krigging_work_space\\kriging_output_AQ_THK_1.tif',
 'D:/GW_data_interpolated\\Well_data_krigging\\Krigging_work_space_250m_entire_michigan\\Krigging_work_space\\kriging_output_AQ_THK_2.tif',
 'D:/GW_data_interpolated\\Well_data_krigging\\Krigging_work_space_250m_entire_michigan\\Krigging_work_space\\kriging_output_H_COND_1.tif',
 'D:/GW_data_interpolated\\Well_data_krigging\\Krigging_work_space_250m_entire_michigan\\Krigging_work_space\\kriging_output_H_COND_2.tif',
 'D:/GW_data_interpolated\\Well_data_krigging\\Krigging_work_space_250m_entire_michigan\\Krigging_work_space\\kriging_output_SWL.tif',
 'D:/GW_data_interpolated\\Well_data_krigging\\Krigging_work_space_250m_entire_michigan\\Krigging_work_space\\kriging_output_TRANSMSV_1.tif',
 'D:/GW_data_interpolated\\Well_data_krigging\\Krigging_work_space_250m_entire_michigan\\Krigging_work_space\\kriging_output_TRANSMSV_2.tif',
 'D:/GW_data_interpol

In [ ]:


import arcpy
from arcpy.sa import *
import os
import glob
import arcpy
import os
import glob
import numpy as np

BASE_PATH=f'D:/GW_data_interpolated/'
SOURCE_mask="Boundary/Michigan_LP_boundary_26990.shp"
#SOURCE_rasters="R:/ram_disk_workspace"

SOURCE_rasters= r'Soil'
MASK_shape=os.path.join(BASE_PATH, SOURCE_mask)
TEMP_path = os.path.join(BASE_PATH, '_temp')
arcpy.env.overwriteOutput=True
FILES=glob.glob(BASE_PATH+SOURCE_rasters+'/*250m.tif')
print(FILES)
run_the_script()